# PyPSA - Python for Power Systems Analysis
pip install pypsa


## Minimal 3-node example of PyPSA power flow
[Source: https://www.pypsa.org/examples/minimal_example_pf.html ]

In [6]:
# make the code as Python 3 compatible as possible
import pypsa
import numpy as np

network = pypsa.Network()

In [7]:
#add three buses
n_buses = 3

for i in range(n_buses):
    network.add("Bus","My bus {}".format(i),
                v_nom=20.) # Nominal voltage

print(network.buses)


attribute  v_nom type    x    y carrier  v_mag_pu_set  v_mag_pu_min  \
My bus 0    20.0       0.0  0.0      AC           1.0           0.0   
My bus 1    20.0       0.0  0.0      AC           1.0           0.0   
My bus 2    20.0       0.0  0.0      AC           1.0           0.0   

attribute  v_mag_pu_max control sub_network  
My bus 0            inf      PQ              
My bus 1            inf      PQ              
My bus 2            inf      PQ              


In [8]:
#add three lines in a ring
for i in range(n_buses):
    network.add("Line","My line {}".format(i),
                bus0="My bus {}".format(i),
                bus1="My bus {}".format((i+1)%n_buses),
                x=0.1, # Series reactance
                r=0.01) # Series resistance
print(network.lines)


attribute      bus0      bus1 type    x     r    g    b  s_nom  \
My line 0  My bus 0  My bus 1       0.1  0.01  0.0  0.0    0.0   
My line 1  My bus 1  My bus 2       0.1  0.01  0.0  0.0    0.0   
My line 2  My bus 2  My bus 0       0.1  0.01  0.0  0.0    0.0   

attribute  s_nom_extendable  s_nom_min  ...  v_ang_min  v_ang_max  \
My line 0             False        0.0  ...       -inf        inf   
My line 1             False        0.0  ...       -inf        inf   
My line 2             False        0.0  ...       -inf        inf   

attribute  sub_network  x_pu  r_pu  g_pu  b_pu  x_pu_eff r_pu_eff  s_nom_opt  
My line 0                0.0   0.0   0.0   0.0       0.0      0.0        0.0  
My line 1                0.0   0.0   0.0   0.0       0.0      0.0        0.0  
My line 2                0.0   0.0   0.0   0.0       0.0      0.0        0.0  

[3 rows x 26 columns]


In [9]:

#add a generator at bus 0
network.add("Generator","My gen",
            bus="My bus 0",
            p_set=100, # Active Power Set Point
            control="PQ") # P,Q,V control strategy for PF, must be “PQ”, “PV” or “Slack”.
print(network.generators)
print()
print(network.generators.p_set)



attribute       bus control type  p_nom  p_nom_extendable  p_nom_min  \
My gen     My bus 0      PQ         0.0             False        0.0   

attribute  p_nom_max  p_min_pu  p_max_pu  p_set  ...  shut_down_cost  \
My gen           inf       0.0       1.0  100.0  ...             0.0   

attribute  min_up_time min_down_time  up_time_before  down_time_before  \
My gen               0             0               1                 0   

attribute  ramp_limit_up  ramp_limit_down  ramp_limit_start_up  \
My gen               NaN              NaN                  1.0   

attribute  ramp_limit_shut_down  p_nom_opt  
My gen                      1.0        0.0  

[1 rows x 28 columns]

My gen    100.0
Name: p_set, dtype: float64


In [10]:
#add a load at bus 1
network.add("Load","My load",
            bus="My bus 1",
            p_set=100) # Active power consumption
print(network.loads)
print()
print(network.loads.p_set)

network.loads.q_set = 100. # Reactive power consumption
print()
print(network.loads.q_set)

attribute       bus type  p_set  q_set  sign
My load    My bus 1       100.0    0.0  -1.0

My load    100.0
Name: p_set, dtype: float64

My load    100.0
Name: q_set, dtype: float64


# Newton-Raphson Power Flow
In the Newton-Raphson power flow we use Newton's. method to determine the voltage magnitude and angle at. each bus in the power system that satisfies power balance.

The objective of a power flow study is to calculate the voltages (magnitude and angle) for a given load, generation, and network condition. Once voltages are known for all buses, line flows and losses can be calculated.

In [11]:
#Do a Newton-Raphson power flow
network.pf()
print("Voltage for lines:")
print(network.lines_t.p0) # Active Power at Bus 0
print()
print("Voltage Angles for buses:")
print(network.buses_t.v_ang*180/np.pi)
print()
print("Voltage Magnitudes for buses:")
print(network.buses_t.v_mag_pu)

INFO:pypsa.pf:Performing non-linear load-flow on AC sub-network SubNetwork 0 for snapshots Index(['now'], dtype='object')
INFO:pypsa.pf:Newton-Raphson solved in 3 iterations with error of 0.000000 in 0.026992 seconds


Voltage for lines:
     My line 0  My line 1  My line 2
now  66.897487 -33.333333 -33.391038

Voltage Angles for buses:
     My bus 0  My bus 1  My bus 2
now       0.0 -0.875939 -0.433813

Voltage Magnitudes for buses:
     My bus 0  My bus 1  My bus 2
now       1.0  0.981199   0.99057
